In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def parse_model_config(path):
    """Parses the yolo-v3 layer configuration file and returns module definitions"""
    file = open(path, 'r')
    lines = file.read().split('\n')
    lines = [x for x in lines if x and not x.startswith('#')]
    lines = [x.rstrip().lstrip() for x in lines] # get rid of fringe whitespaces
    module_defs = []
    for line in lines:
        if line.startswith('['): # This marks the start of a new block
            module_defs.append({})
            module_defs[-1]['type'] = line[1:-1].rstrip()
            if module_defs[-1]['type'] == 'convolutional':
                module_defs[-1]['batch_normalize'] = 0
        else:
            key, value = line.split("=")
            value = value.strip()
            module_defs[-1][key.rstrip()] = value.strip()

    return module_defs


def save_model_config(model_defs, path):
    """Saves the yolo-v3 layer configuration file"""
    with open(path, 'w') as writer:
        
        for block in model_defs:
            writer.write('['+ block['type'] +']'+'\n')
            [writer.write(str(k)+'='+str(v)+'\n') for k,v in block.items() if k != 'type']
            writer.write('\n')
    return path

            
def save_data_config(data_config, path):
    """Saves the yolo-v3 data configuration file"""
    with open(path, 'w') as writer:
        [writer.write(str(k)+'='+str(v)+'\n') for k,v in data_config.items()]
    return path


def inject_model_config(dataset, model_config, hyperparams):
    for i, block in enumerate(model_config):        
        if block['type'] == 'net':
            block['learning_rate'] = hyperparams['lr']
            block['batch'] = hyperparams['batch']
            block['subdivisions'] = hyperparams['subdivisions']
            block['burn_in'] = len(dataset._images.items())//(hyperparams['gpus'] * hyperparams['batch'])
            block['max_batches'] = len(dataset._images.items())//(hyperparams['gpus'] * hyperparams['batch']) * hyperparams['epochs']
        elif block['type'] == 'yolo':
            block['classes'] = len(dataset.category_names)
            model_config[i-1]['filters'] = (len(dataset.category_names)+5)*3
    return model_config


def inject_data_config(dataset, data_config):
    data_config['train'] = dataset.darknet_manifast
    data_config['classes'] = len(dataset.category_names)
    ## TODO: Add Validation Set
    data_config['valid'] = dataset.darknet_manifast
    data_config['names'] = dataset.names_config
    backup_path = os.path.abspath(os.path.join(dataset.output_path, os.pardir, 'backup'))
    os.makedirs(backup_path, exist_ok = True)
    data_config['backup'] = os.path.abspath(os.path.join(dataset.output_path, os.pardir, 'backup'))
    num_gpus = int(dataset.parse_nvidia_smi()['Attached GPUs'])
    data_config['gpus'] = ','.join(str(i) for i in range(num_gpus))
    
    
    return data_config
            

def parse_data_config(path):
    """Parses the data configuration file"""
    options = dict()
    options['gpus'] = '0,1'
    with open(path, 'r') as fp:
        lines = fp.readlines()
    for line in lines:
        line = line.strip()
        if line == '' or line.startswith('#'):
            continue
        key, value = line.split('=')
        options[key.strip()] = value.strip()
    return options



In [3]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO
import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle
import pandas as pd
from subprocess import Popen,PIPE,STDOUT,call 
from utils import datasets

BDD100K_DIRECTORY = os.path.join('/media/dean/datastore1/datasets/BerkeleyDeepDrive', 'bdd100k')
WORKING_DIRECTORY = '/media/dean/datastore1/datasets/darknet'

TRAINERS_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'trainers')
ANNOTATIONS_FILE = os.path.join(BDD100K_DIRECTORY, 'labels/bdd100k_labels_images_train.json')
COCO_ANNOTATIONS_FILE = os.path.join('/media/dean/datastore1/datasets/road_coco/darknet/data/coco/annotations/instances_train2014.json')

BASE_DATA_CONFIG = os.path.join(WORKING_DIRECTORY, 'cfg', 'bdd100k.data')
BASE_MODEL_CONFIG = os.path.join(WORKING_DIRECTORY, 'cfg', 'yolov3-bdd100k.cfg')

In [4]:
## Prepare Dataset ##
coco_set = datasets.DataFormatter(annotations_list = COCO_ANNOTATIONS_FILE, input_format = datasets.Format.coco,
                                 output_path = os.path.join(WORKING_DIRECTORY,'data'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/coco_dataset/images/train2014/')


bdd_set = datasets.DataFormatter(annotations_list = ANNOTATIONS_FILE, input_format = datasets.Format.bdd,
                                 output_path = os.path.join(WORKING_DIRECTORY,'data'),
                                 trainer_prefix = 'COCO_train2014_0000', 
                                 s3_bucket = 'kache-scalabel/bdd100k/images/100k/train/')

bdd_set.merge(coco_set, include=['person'], reject_new_categories = True)

Length of COCO Images 82081
Length of COCO Annotations: 604907
loading annotations into memory...
Done (t=2.87s)
creating index...
index created!
########## DATASET DISTRIBUTION: ############

AIRPLANE | Annotations: 3833  | Images:  2243
APPLE | Annotations: 4308  | Images:  1171
BACKPACK | Annotations: 6200  | Images:  3924
BANANA | Annotations: 6912  | Images:  1618
BASEBALL BAT | Annotations: 2400  | Images:  1804
BASEBALL GLOVE | Annotations: 2689  | Images:  1884
BEAR | Annotations: 903  | Images:  668
BED | Annotations: 2905  | Images:  2539
BENCH | Annotations: 6751  | Images:  3844
BICYCLE | Annotations: 4955  | Images:  2287
BIRD | Annotations: 7290  | Images:  2241
BOAT | Annotations: 7590  | Images:  2098
BOOK | Annotations: 17315  | Images:  3734
BOTTLE | Annotations: 16983  | Images:  5968
BOWL | Annotations: 10064  | Images:  5028
BROCCOLI | Annotations: 4927  | Images:  1340
BUS | Annotations: 4327  | Images:  2791
CAKE | Annotations: 4551  | Images:  2080
CAR | Annotat